In [ ]:
from google.colab import files  # upload our file, no need to download it anymore
uploaded = files.upload()

Saving data_loading.py to data_loading (1).py


In [ ]:
from data_loading import get_data_BIG_WGAN  # get all our data as usual
(poland_images, poland_labels), (china_images, china_labels), (egypt_images, egypt_labels), (egypt_mixed_images_1180, egypt_mixed_labels_1180), (egypt_mixed_images_2360, egypt_mixed_labels_2360) = get_data_BIG_WGAN()

downloading a file with egypt 1180 samples (Main + BIG-WGAN) and poland, china


Downloading...
From (original): https://drive.google.com/uc?id=1PY29AhKzMo_msc0-8nc0oXnyWQHpbjWO
From (redirected): https://drive.google.com/uc?id=1PY29AhKzMo_msc0-8nc0oXnyWQHpbjWO&confirm=t&uuid=29d2639e-def3-47b7-a306-ff406c9743cf
To: /content/data_zipped
100%|██████████| 47.6M/47.6M [00:01<00:00, 35.1MB/s]


Poland: images (256, 128, 128), labels (256,)
China: images (635, 128, 128), labels (635,)
Egypt: images (780, 128, 128), labels (780,)
unbalanced, unshuffled,
0 for malignant, 1 for benign,
Poland label distribution:
  Label 0: 102
  Label 1: 154

China label distribution:
  Label 0: 435
  Label 1: 200

Egypt label distribution:
  Label 0: 210
  Label 1: 570

Egypt mixed with GAN:
  Label 0: 410
  Label 1: 770

Egypt mixed with GAN and FLIPPED:
  Label 0: 820
  Label 1: 1540
returns poland images(TEST THIS), china images, egypt images, egypt mixed images 1180(TRAIN THIS), egypt mixed images 2360(TRAIN THIS)


In [ ]:
# all the fun libraries
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
from torch.utils.data import Dataset
import os
!pip install timm torchvision

In [ ]:
import torch.nn as nn
from torchvision.models.vision_transformer import VisionTransformer

# had to look up how to use this
class ViTBinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.vit = VisionTransformer(
            image_size=128,
            patch_size=16,  # 16x16 patches
            num_layers=12,
            num_heads=12, # 12 heads
            hidden_dim=768,    # 768 = 12x64 , standard pick
            mlp_dim=3072,
            num_classes=1     # one class because of BCE ( 0 or 1 of the one class)
        )
        self.vit.conv_proj = nn.Conv2d(1, 768, kernel_size=16, stride=16)

    def forward(self, x):
        out = self.vit(x)
        return out.squeeze(1)


In [ ]:
# get the data set that we train and test on
egypt_x = np.load('/content/data_unzipped/Datasets/Egypt Hospital - Breast Ultrasound/Egypt Hospital 128x128/Egypt Hospital_US_Breast_Images.npy')
egypt_y = np.load('/content/data_unzipped/Datasets/Egypt Hospital - Breast Ultrasound/Egypt Hospital 128x128/Egypt Hospital_US_Breast_Labels.npy')
poland_x = np.load('/content/data_unzipped/Datasets/Polish Hospital - Breast Ultrasound 128x128/Poland_Medical_Centers_US_Breast_Images.npy')
poland_y = np.load('/content/data_unzipped/Datasets/Polish Hospital - Breast Ultrasound 128x128/Poland_Medical_Centers_US_Breast_Labels.npy')

In [ ]:
# lazy reassignment from the code in the second block
x_train = egypt_mixed_images_2360
y_train = egypt_mixed_labels_2360
x_test = poland_x
y_test = poland_y

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(egypt_x, egypt_y, test_size=0.2, random_state=42) # split 80/20

"""
train_dataset = GrayscaleImageDataset(x_train, y_train)
test_dataset = GrayscaleImageDataset(x_test, y_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape  # looks good

((2360, 128, 128), (256, 128, 128), (2360,), (256,))

In [ ]:
# check if cuda is ready, show device
torch.cuda.is_available()
print(torch.cuda.get_device_name())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Tesla T4
cuda


In [ ]:
import time
model = ViTBinaryClassifier().to(device)  # classifier
criterion = nn.BCEWithLogitsLoss()  # loss , this one is different from CrossEntropy, I think crossentropy needs 2 classes?
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5) # optimizer

begin = time.time()
for epoch in range(20):
    model.train() # train model
    for images, labels in train_loader: # images and label tensored to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() # grad 0
        outputs = model(images) # classify
        loss = criterion(outputs, labels) # get loss(BCE)
        loss.backward()
        optimizer.step()  # step

    print(f"Epoch {epoch} Loss: {loss.item():.4f}")

end = time.time()
print(f"Training time: {end - begin} seconds")

Epoch 0 Loss: 0.6844
Epoch 1 Loss: 0.6020
Epoch 2 Loss: 0.6628
Epoch 3 Loss: 0.5301
Epoch 4 Loss: 0.4880
Epoch 5 Loss: 0.6193
Epoch 6 Loss: 0.4047
Epoch 7 Loss: 0.3757
Epoch 8 Loss: 0.7721
Epoch 9 Loss: 0.2382
Epoch 10 Loss: 0.2886
Epoch 11 Loss: 0.3291
Epoch 12 Loss: 0.5183
Epoch 13 Loss: 0.2863
Epoch 14 Loss: 0.1355
Epoch 15 Loss: 0.1581
Epoch 16 Loss: 0.2042
Epoch 17 Loss: 0.3213
Epoch 18 Loss: 0.3340
Epoch 19 Loss: 0.2387
Training time: 566.2656383514404 seconds


In [ ]:
from sklearn.metrics import accuracy_score

model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = (torch.sigmoid(outputs) > 0.5).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print("Test Accuracy:", accuracy_score(all_labels, all_preds))


Test Accuracy: 0.6015625


Big cross domain tests  
(20 epoch):    
train egypt test poland 60.5%  
train egypt 1180 test poland 57.4%  
train egypt 2360 test poland 60
(50 epoch):  





with 25 epochs and very small train/test (prof suggestion):  
egypt 100 real, test on egypt  70%  
egypt 100 real, test on poland 55%  
egypt 100 real + 100 fake, test on poland  55%  
egypt 100 real + 75 fake, test on poland 55%  
egypt 100 real + 50 fake, test on poland  58%  
egypt 100 real + 25 fake, test on poland 50%

with 50 epochs: \
egypt train and egypt test 74.3%\
egypt train and poland test 61.7%\
poland train and poland test 67.3%\
poland train and egypt test 54%\
egypt_25WGAN train poland test 61.7%\
egypt_50WGAN train poland test 55.4% \
egypt_75WGAN train poland test 60.1% \
egypt_100WGAN train poland test 60.1% \
egypt_25CGAN train poland test 60.1%\
egypt_50CGAN train poland test 62.8%\
egypt_75CGAN train poland test 60.1%\
egypt_100CGAN train poland test 60.1%\
egypt_1080 train poland test 63.28%\
egypt_2360 train poland test 62.1%\

with 25 epochs:   
egypt train poland test 61.7%   
egypt_mixed_1180 train poland test
add only small portions

